In [3]:
import string

FILE = "american-english.txt"
ALLOWABLE_CHARACTERS = set(string.ascii_letters)
ALLOWED_ATTEMPTS = 6
WORD_LENGTH = 5

In [4]:
wordlist = [word.strip() for word in open(FILE, "r").readlines()]

#creating a set of valid words according to the rules of Wordle
WORDS = {
    word.lower()
    for word in wordlist    
    if len(word) == WORD_LENGTH and set(word) < ALLOWABLE_CHARACTERS
    }
WORDS

{'loose',
 'crowd',
 'gusto',
 'modem',
 'eight',
 'dryly',
 'nosey',
 'worse',
 'below',
 'harry',
 'purer',
 'retro',
 'moldy',
 'lorry',
 'mauve',
 'icily',
 'newly',
 'recap',
 'tryst',
 'chili',
 'turbo',
 'wheel',
 'funny',
 'guise',
 'nudge',
 'proxy',
 'leapt',
 'suave',
 'dowry',
 'gloat',
 'maxim',
 'stunt',
 'movie',
 'sheen',
 'knock',
 'ideal',
 'joust',
 'audio',
 'curry',
 'druid',
 'flyer',
 'rupee',
 'spree',
 'viper',
 'saucy',
 'pushy',
 'duchy',
 'avail',
 'extol',
 'stint',
 'youth',
 'stung',
 'kneed',
 'basal',
 'bulge',
 'exact',
 'jumpy',
 'spoil',
 'unset',
 'belle',
 'locus',
 'beret',
 'minty',
 'cigar',
 'baggy',
 'mania',
 'minim',
 'smack',
 'canal',
 'using',
 'inter',
 'proud',
 'grief',
 'chief',
 'spice',
 'cache',
 'bleak',
 'grace',
 'spicy',
 'idiot',
 'eaten',
 'space',
 'mossy',
 'fable',
 'opium',
 'plumb',
 'affix',
 'dread',
 'cairn',
 'beefy',
 'there',
 'court',
 'glass',
 'teach',
 'hotly',
 'utile',
 'pixel',
 'giver',
 'paper',
 'stilt',


In [5]:
from collections import Counter
from itertools import chain

#Creating a Counter Object that counts the number of occurences
#of each letter used across all valid Wordle words

LETTER_COUNTER = Counter(chain.from_iterable(WORDS))

LETTER_COUNTER


Counter({'l': 719,
         'o': 754,
         's': 669,
         'e': 1233,
         'c': 477,
         'r': 899,
         'w': 195,
         'd': 393,
         'g': 311,
         'u': 467,
         't': 729,
         'm': 316,
         'i': 671,
         'h': 389,
         'y': 425,
         'n': 575,
         'b': 281,
         'a': 979,
         'p': 367,
         'v': 153,
         'f': 230,
         'x': 37,
         'k': 210,
         'j': 27,
         'z': 40,
         'q': 29})

In [6]:
#finding the total occurances of all letters
total = len(WORDS)*WORD_LENGTH
LETTER_FREQUENCY = {character: value / total 
                    for character, value in LETTER_COUNTER.items()}

LETTER_FREQUENCY

{'l': 0.06211663066954644,
 'o': 0.0651403887688985,
 's': 0.057796976241900645,
 'e': 0.10652267818574514,
 'c': 0.04120950323974082,
 'r': 0.07766738660907127,
 'w': 0.016846652267818573,
 'd': 0.033952483801295896,
 'g': 0.026868250539956805,
 'u': 0.04034557235421166,
 't': 0.06298056155507559,
 'm': 0.027300215982721383,
 'i': 0.057969762419006476,
 'h': 0.033606911447084234,
 'y': 0.0367170626349892,
 'n': 0.04967602591792657,
 'b': 0.02427645788336933,
 'a': 0.08457883369330453,
 'p': 0.031706263498920084,
 'v': 0.013218142548596112,
 'f': 0.019870410367170625,
 'x': 0.0031965442764578834,
 'k': 0.01814254859611231,
 'j': 0.002332613390928726,
 'z': 0.0034557235421166306,
 'q': 0.002505399568034557}

In [7]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += LETTER_FREQUENCY[char]
    return score / (WORD_LENGTH - len(set(word)) + 1)

In [8]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [9]:
def input_word():
    while True:
        word = input("Input the word you entered> ")
        if len(word) == WORD_LENGTH and word.lower() in WORDS:
            break
    return word.lower()

def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [10]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [11]:
def solve():
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        display_word_table(sort_by_word_commonality(possible_words)[:15])
        word = input_word()
        response = input_response()
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [12]:
solve()

Attempt 1 with 2315 possible words
alert      | 0.39 
later      | 0.39 
alter      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
raise      | 0.38 
arise      | 0.38 
renal      | 0.38 
learn      | 0.38 
saner      | 0.38 
snare      | 0.38 
slate      | 0.37 
least      | 0.37 
steal      | 0.37 


Input the word you entered>  gayer


Type the color-coded reply from Wordle:
 G for Green
 Y for Yellow
 ? for Gray


KeyboardInterrupt: Interrupted by user